In [1]:
import pandas as pd
import re


# pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
df = pd.read_csv('resumes_ttl.csv')

предварительно дубли почищены

In [3]:
df.head()

,link,name,gender,age,path1,path2,skills,languages,salary,experience,edu_degree,uni
0,https://hh.ru/resume/f6343d3e000364b04e0039ed1...,Аналитик,Мужчина,29 лет,"Банки, инвестиции, лизинг",['Аналитик'],"['Креативность', 'Работа в команде', 'Грамотна...","['Русский — Родной', 'Английский — C1 — Продви...",0,Опыт работы 9 лет 3 месяца,Высшее образование,['Государственный университет по землеустройст...
1,https://hh.ru/resume/a460bf600008813d1e0039ed1...,Аналитик-переводчик,Мужчина,26 лет,"Информационные технологии, интернет, телеком",['Аналитик'],"['Работоспособность', 'SWOT анализ', 'Перевод'...","['Русский — Родной', 'Английский — C2 — В сове...",70 000 руб.,Опыт работы 1 год 3 месяца,Высшее образование (Бакалавр),['USEEFaculty of Legal and Business Studies Dr...
2,https://hh.ru/resume/1d5acdda0008d6a0580039ed1...,Аналитик,Мужчина,25 лет,"Банки, инвестиции, лизинг",['Аналитик'],"['SQL', 'DWH', 'Бизнес-анализ', 'Системный ана...","['Русский — Родной', 'Английский — B1 — Средний']",0,Опыт работы 2 года 3 месяца,Высшее образование (Магистр),['Московский государственный технический униве...
3,https://hh.ru/resume/f068d4e200064239590039ed1...,Аналитик,Мужчина,26 лет,"Информационные технологии, интернет, телеком","['Тестирование', 'Управление проектами', 'Анал...","['Аналитическое мышление', 'Работа в команде',...","['Русский — Родной', 'Английский — A1 — Началь...",180 000 руб.,Опыт работы 2 года 4 месяца,Высшее образование (Бакалавр),['Тамбовский государственный технический униве...
4,https://hh.ru/resume/1ea9beb30009320c280039ed1...,Аналитик,Мужчина,0,0,0,"['Бумажный документооборот', 'Документооборот'...",0,0,Опыт работы 5 лет 5 месяцев,0,0


#### Возраст -> int

In [5]:
df['age'].head()

0    29 лет
1    26 лет
2    25 лет
3    26 лет
4         0
Name: age, dtype: object

In [6]:
df['age'] = df['age'].apply(lambda x: int(re.search(r'\d+', x)[0]))

#### Пол -> 3 категории

In [7]:
df['gender'].value_counts()

Мужчина           11485
Женщина            2837
Male               1191
Female              346
Пол не указан         5
Gender unknown        2
0                     1
Name: gender, dtype: int64

In [8]:
df.loc[(df['gender'] == 'Мужчина') | (df['gender'] == 'Male'), 'gender'] = 'M'
df.loc[(df['gender'] == 'Женщина') | (df['gender'] == 'Female'), 'gender'] = 'F'
df.loc[(df['gender'] == 'Пол не указан') | (df['gender'] == 'Gender unknown') | (df['gender'] == '0'), 'gender'] = 'unknown'

In [9]:
df['gender'].value_counts()

M          12676
F           3183
unknown        8
Name: gender, dtype: int64

#### ЗП -> int

In [10]:
df['salary'].value_counts().index

Index(['0', '50 000 руб.', '100 000 руб.', '80 000 руб.', '60 000 руб.',
       '70 000 руб.', '40 000 руб.', '150 000 руб.', '120 000 руб.',
       '30 000 руб.',
       ...
       '8 300 EUR', '370 000 RUB', '29 000 руб.', '3 200 USD', '10 руб.',
       '2 700 USD', '211 000 руб.', '15 000 USD', '72 500 руб.', '800 EUR'],
      dtype='object', length=352)

In [12]:
def currency(i):
    try:
        if ('руб.' in i) or ('RUB' in i):
            return int(i.replace('руб.', '').replace('RUB', '').replace('\u2009', '').rsplit("\xa0", 1)[0])
        elif ('EUR' in i):
            return int(i.replace('EUR', '').replace('\u2009', '').rsplit("\xa0", 1)[0]) * 82.5
        elif ('USD' in i):
            return int(i.replace('USD', '').replace('\u2009', '').rsplit("\xa0", 1)[0]) * 73.5
        elif ('AZN' in i) or ('KGS' in i):
            return 0
        else:
            return i
    except:
        return i

In [13]:
df['salary'] = df['salary'].apply(lambda x:  currency(x))

for i in list(df['salary'].value_counts().index):
    if type(i) != int:
        print(i)

In [15]:
df['salary'] = pd.to_numeric(df['salary'])

In [28]:
df.to_csv('res_prim_clean.csv', index=False)